<a href="https://colab.research.google.com/github/KiranEC11/LSTM/blob/main/LSTM_Next_n_character_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
import keras

In [3]:
import tensorflow as tf

# Check if a GPU is available
if tf.test.is_gpu_available():
    print("GPU available")
else:
    print("GPU not available")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU available


In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import utils as np_utils

### load txt file

In [9]:
# filename = 'Macbeth.txt'
# with open(filename, 'r') as f:
#     lines = f.readlines()

In [1]:
filename = 'Macbeth.txt'
text = (open(filename).read()).lower()

In [2]:
text[:1000]

"the tragedie of macbeth\n\nactus primus. scoena prima.\n\nthunder and lightning. enter three witches.\n\n  1. when shall we three meet againe?\nin thunder, lightning, or in raine?\n  2. when the hurley-burley's done,\nwhen the battaile's lost, and wonne\n\n   3. that will be ere the set of sunne\n\n   1. where the place?\n  2. vpon the heath\n\n   3. there to meet with macbeth\n\n   1. i come, gray-malkin\n\n   all. padock calls anon: faire is foule, and foule is faire,\nhouer through the fogge and filthie ayre.\n\nexeunt.\n\n\nscena secunda.\n\nalarum within. enter king, malcome, donalbaine, lenox, with\nattendants, meeting a bleeding captaine.\n\n  king. what bloody man is that? he can report,\nas seemeth by his plight, of the reuolt\nthe newest state\n\n   mal. this is the serieant,\nwho like a good and hardie souldier fought\n'gainst my captiuitie: haile braue friend;\nsay to the king, the knowledge of the broyle,\nas thou didst leaue it\n\n   cap. doubtfull it stood,\nas two spen

### preprocessing steps

In [3]:
text  = text.replace('\n','').replace('\n','')

In [4]:
text[:1000]

"the tragedie of macbethactus primus. scoena prima.thunder and lightning. enter three witches.  1. when shall we three meet againe?in thunder, lightning, or in raine?  2. when the hurley-burley's done,when the battaile's lost, and wonne   3. that will be ere the set of sunne   1. where the place?  2. vpon the heath   3. there to meet with macbeth   1. i come, gray-malkin   all. padock calls anon: faire is foule, and foule is faire,houer through the fogge and filthie ayre.exeunt.scena secunda.alarum within. enter king, malcome, donalbaine, lenox, withattendants, meeting a bleeding captaine.  king. what bloody man is that? he can report,as seemeth by his plight, of the reuoltthe newest state   mal. this is the serieant,who like a good and hardie souldier fought'gainst my captiuitie: haile braue friend;say to the king, the knowledge of the broyle,as thou didst leaue it   cap. doubtfull it stood,as two spent swimmers, that doe cling together,and choake their art: the mercilesse macdonwald(

## prepare train dataset

In [5]:
# store unique words
unique_chars  = sorted(list(set(text)))

In [12]:
len(unique_chars)

42

In [13]:
unique_chars

[' ',
 '!',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '1',
 '2',
 '3',
 ':',
 ';',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [6]:
## char to int map
char_to_int = {}

## int to char map
int_to_char = {}

## lets update both the above maps

for i,char in enumerate(unique_chars):
    char_to_int.update({char:i})
    int_to_char.update({i:char})


In [15]:
char_to_int['a']

17

In [16]:
int_to_char[17]

'a'

its done

In [7]:
## now prepare the train dataset
"""
collect sequences and next labels.
timesteps = 50
"""
timestep = 50

X = []
y = []

for i in range(0, len(text)-timestep, 1):
    sequence = text[i:i+timestep]

    X.append([char_to_int[char] for char in sequence])
    y.append(char_to_int[text[i+timestep]])

In [50]:
# X[0] , y[0]

## reshaping X

In [18]:
"""
X shape should be = [samples, timsteps, features]
features = number of outputs --> one character means features =1
"""

## reshaping
X_modified =  np.reshape(X, (len(X), timestep, 1))

## normalising
X_modified = X_modified/float(len(unique_chars))

In [19]:
##y one hot encoding
y_modified = np_utils.to_categorical(y)

In [20]:
y_modified[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

## LSTM architecture

In [21]:
model = Sequential()
model.add(LSTM(300, input_shape = (X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam')



## fitting the model and generating characters

In [23]:
model.fit(X_modified, y_modified, epochs = 100, batch_size = 30, verbose=True)

Epoch 1/100
3233/3233 [==============================] - 32s 10ms/step - loss: 2.6610
Epoch 2/100
3233/3233 [==============================] - 33s 10ms/step - loss: 2.5007
Epoch 3/100
3233/3233 [==============================] - 33s 10ms/step - loss: 2.3685
Epoch 4/100
3233/3233 [==============================] - 32s 10ms/step - loss: 2.2635
Epoch 5/100
3233/3233 [==============================] - 32s 10ms/step - loss: 2.1780
Epoch 6/100
3233/3233 [==============================] - 33s 10ms/step - loss: 2.1114
Epoch 7/100
3233/3233 [==============================] - 32s 10ms/step - loss: 2.0394
Epoch 8/100
3233/3233 [==============================] - 33s 10ms/step - loss: 1.9802
Epoch 9/100
3233/3233 [==============================] - 32s 10ms/step - loss: 1.9281
Epoch 10/100
3233/3233 [==============================] - 32s 10ms/step - loss: 1.8785
Epoch 11/100
3233/3233 [==============================] - 33s 10ms/step - loss: 1.8252
Epoch 12/100
3233/3233 [============================

In [24]:
## generate char sequence

start_index = np.random.randint(0,len(X)-1)

sequence = X[start_index]
print(start_index)
char_seq = [int_to_char[inte] for inte in sequence]

prediction = ''
for i in range(10):

    ## reshape sequence
    x = np.reshape(sequence, (1, timestep,1))
    x = x / float(len(unique_chars))

    # predict
    y_pred_int = np.argmax(model.predict(x, verbose=0))
    char_out = int_to_char[y_pred_int]
    prediction += char_out

    ## update sequence for next char prediction
    sequence.append(y_pred_int)
    sequence = sequence[-timestep:]

print(f'input string: {"".join(char_seq)}')
print(f'generated string: {prediction}')

6727
input string: as breath into the winde.would they had stay'd   b
generated string: anq. whos 


In [8]:
"".join([int_to_char[inte] for inte in X[6727+10][-10:]])

'anq. were '

## Results:

### Generate next 10 characters after the below sequence:

#### sequence: "as breath into the winde.would they had stay'd   b"

#### true: "anq. were"

#### predicted: "anq. whos"